In [34]:
%matplotlib inline
from IPython.display import Image
import numpy as np
import matplotlib.pyplot as plt

import openmc

In [52]:
# Instantiate Nuclides
u235 = openmc.Nuclide('U235')
he3 = openmc.Nuclide('He3')

# Create materials
fuel = openmc.Material(name='fuel')
fuel.set_density('g/cm3', 10.31341)
fuel.add_nuclide(u235, 1.0)

air = openmc.Material(name='fuel')
air.set_density('g/cm3', 10.31341)
air.add_nuclide(he3, 1.0)

materials_file = openmc.Materials((fuel, air))
materials_file.default_xs = '71c'
materials_file.export_to_xml()

radius = 0.5
settings_file = openmc.Settings()
settings_file.batches = 100
settings_file.inactive = 10
settings_file.particles = 5000
# Create geometry
sphere = openmc.Sphere(R=radius)
min_x = openmc.XPlane(x0=-radius, boundary_type='vacuum') 
max_x = openmc.XPlane(x0=+radius, boundary_type='vacuum') 
min_y = openmc.YPlane(y0=-radius, boundary_type='vacuum') 
max_y = openmc.YPlane(y0=+radius, boundary_type='vacuum') 
min_z = openmc.ZPlane(z0=-radius, boundary_type='vacuum')
max_z = openmc.ZPlane(z0=+radius, boundary_type='vacuum') 

# Create Universe
universe = openmc.Universe(name='Universe')

fuel_cell = openmc.Cell(name='fuel')
fuel_cell.fill = fuel
fuel_cell.region = -sphere
universe.add_cell(fuel_cell)

air_cell = openmc.Cell(name='air')
air_cell.fill = air
air_cell.region = +sphere
universe.add_cell(air_cell)

# Create root cell
root_cell = openmc.Cell(name='root_cell')
root_cell.fill = universe
root_cell.region = +min_x & -max_x & +min_y & -max_y & +min_z & -max_z
root_universe = openmc.Universe(universe_id=0, name='root universe')
root_universe.add_cell(root_cell)

geometry = openmc.Geometry()
geometry.root_universe = root_universe
geometry.export_to_xml()

# Create source
bounds = [-radius, -radius, -radius, radius, radius, radius]
uniform_dist = openmc.stats.Box(bounds[:3], bounds[3:], only_fissionable=True)
settings_file.source = openmc.source.Source(space=uniform_dist)
settings_file.export_to_xml()

# Run the calculation
openmc.run()

# Load the statepoint file
sp = openmc.StatePoint('statepoint.100.h5')
print(openmc.StatePoint.k_combined)


                               %%%%%%%%%%%%%%%
                          %%%%%%%%%%%%%%%%%%%%%%%%
                       %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                     %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                  %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                   %%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                ###############      %%%%%%%%%%%%%%%%%%%%%%%%
               ##################     %%%%%%%%%%%%%%%%%%%%%%%
               ###################     %%%%%%%%%%%%%%%%%%%%%%%
               ####################     %%%%%%%%%%%%%%%%%%%%%%
               #####################     %%%%%%%%%%%%%%%%%%%%%
               ######################     %%%%%%%%%%%%%%%%%%%%
               #######################     %%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%

UnicodeDecodeError: 'ascii' codec can't decode byte 0x9e in position 1: ordinal not in range(128)